# TP1 - Regresión
Notebook inicial para el trabajo práctico.

In [3]:
# 1. Importación de librerías necesarias
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler   # u otros scalers
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
# 2. Carga de datos. Se carga el dataset que contiene los viajes de taxi.
file_path= 'uber_fares.csv'
df = pd.read_csv(file_path)

# visualizacion de algunos datos
df.head()

,key,date,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


#### Contexto  
El proyecto trata sobre **Uber Inc.**, la compañía de taxis más grande del mundo. En este trabajo, nuestro objetivo es **predecir la tarifa de futuros viajes**.  

Uber brinda servicio a millones de clientes cada día, por lo que gestionar adecuadamente sus datos es clave para desarrollar nuevas estrategias de negocio y obtener mejores resultados.  

### Variables del conjunto de datos  

**Variables explicativas:**  
- **key**: identificador único de cada viaje.  
- **pickup_datetime**: fecha y hora en que se inició el viaje.  
- **passenger_count**: cantidad de pasajeros en el vehículo (dato ingresado por el conductor).  
- **pickup_longitude**: longitud del punto de inicio del viaje.  
- **pickup_latitude**: latitud del punto de inicio del viaje.  
- **dropoff_longitude**: longitud del punto de destino.  
- **dropoff_latitude**: latitud del punto de destino.  

**Variable objetivo (target):**  
- **fare_amount**: costo del viaje en dólares.  

In [ ]:
# 3. Análisis descriptivo de las variables
# Comentario: Se realiza un análisis inicial para comprender el comportamiento de cada variable.
# - Descripción estadística (df.describe())
# - Visualización de valores únicos y rangos


# Columnas, ¿cuáles son variables numéricas y cuales variables categóricas?
df.columns


Index(['key', 'date', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

# - Crear variables como: 
distancia del viaje, horario (puede ser en formato como explicaron en clase), día de la semana, semana del año, ambas podrían ser cualitativas ordinales.
Distancia entre origen y destino:
Distancia Euclidiana (en grados, no es realista pero es rápida)
Distancia Haversine (más precisa para lat/lon: kilómetros o metros)
Diferencia absoluta de longitudes/latitudes (puede ayudar)
Duración del viaje:
estimar duración usando distancia y velocidad promedio
Características temporales del viaje
Extrae del campo pickup_datetime:
Hora del día (0-23): ¿Influye el horario en la tarifa?
Día de la semana (lunes a domingo): ¿Hay diferencias entre días laborales y fines de semana?
Mes o estación del año: ¿Hay estacionalidad?
¿Es hora pico? (por ejemplo, 7-9 AM y 5-7 PM): variable booleana
¿Es fin de semana? (booleana)
Variables de interacción
Distancia × cantidad de pasajeros: para ver si viajes largos con muchos pasajeros tienen distinta tarifa por pasajero.
Hora × día de la semana: para captar efectos combinados de horario y día.


- Identificación de valores atípicos (outliers) y datos faltantes (missing values)

In [ ]:
def distancia_euclidiana(lat1, lon1, lat2, lon2):
    """
    Calcula la distancia euclidiana entre dos puntos (lat1, lon1) y (lat2, lon2).
    La medida es en grados, porque no considera la curvatura de la tierra.
    """
    return np.sqrt((lat2 - lat1) ** 2 + (lon2 - lon1) ** 2)

# Distancia Manhattan
def distancia_absoluta(lat1, lon1, lat2, lon2):
    """
    Calcula la suma de las diferencias absolutas entre latitud y longitud.
    Es decir, la distancia Manhattan en coordenadas.
    """
    return np.abs(lat2 - lat1) + np.abs(lon2 - lon1)

In [12]:
df["dist_euclidiana"] = distancia_euclidiana(
    df["pickup_latitude"], df["pickup_longitude"],
    df["dropoff_latitude"], df["dropoff_longitude"]
)
df["dist_absoluta"] = distancia_absoluta(
    df["pickup_latitude"], df["pickup_longitude"],
    df["dropoff_latitude"], df["dropoff_longitude"]
)

In [14]:
print(df.head())  # Muestra las primeras 5 filas de todo el df
print(df.tail())  # Muestra las últimas 5 filas

        key                           date  fare_amount  \
0  24238194    2015-05-07 19:52:06.0000003          7.5   
1  27835199    2009-07-17 20:04:56.0000002          7.7   
2  44984355   2009-08-24 21:45:00.00000061         12.9   
3  25894730    2009-06-26 08:22:21.0000001          5.3   
4  17610152  2014-08-28 17:47:00.000000188         16.0   

           pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06 UTC        -73.999817        40.738354   
1  2009-07-17 20:04:56 UTC        -73.994355        40.728225   
2  2009-08-24 21:45:00 UTC        -74.005043        40.740770   
3  2009-06-26 08:22:21 UTC        -73.976124        40.790844   
4  2014-08-28 17:47:00 UTC        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count  dist_euclidiana  \
0         -73.999512         40.723217                1         0.015140   
1         -73.994710         40.750325                1         0.022103   
2         -73.962565      

In [13]:
df[["dist_euclidiana", "dist_absoluta", "fare_amount"]].head(10)

,dist_euclidiana,dist_absoluta,fare_amount
0,0.015140,0.015442,7.5
1,0.022103,0.022455,7.7
2,0.053109,0.074355,12.9
3,0.016528,0.023313,5.3
4,0.051031,0.065221,16.0
5,0.000000,0.000000,4.9
6,0.120825,0.170584,24.5
7,0.000000,0.000000,2.5
8,0.027623,0.029763,9.7
9,0.050496,0.071250,12.5


In [16]:
print(df[["dist_euclidiana", "dist_absoluta"]].describe())

       dist_euclidiana  dist_absoluta
count    199999.000000  199999.000000
mean          0.256968       0.328972
std           9.216476      11.627817
min           0.000000       0.000000
25%           0.012432       0.015863
50%           0.021489       0.027697
75%           0.038335       0.050330
max        2434.714983    3270.155787


In [25]:
# Distancia haversine (en km)
def haversine(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en km
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

In [27]:
df['dist_haversine'] = haversine(
    df['pickup_latitude'], df['pickup_longitude'],
    df['dropoff_latitude'], df['dropoff_longitude'])

In [28]:
df[["dist_euclidiana", "dist_absoluta", "dist_haversine", "fare_amount"]].head(10)

,dist_euclidiana,dist_absoluta,dist_haversine,fare_amount
0,0.015140,0.015442,1.683323,7.5
1,0.022103,0.022455,2.457590,7.7
2,0.053109,0.074355,5.036377,12.9
3,0.016528,0.023313,1.661683,5.3
4,0.051031,0.065221,4.475450,16.0
5,0.000000,0.000000,0.000000,4.9
6,0.120825,0.170584,11.731015,24.5
7,0.000000,0.000000,0.000000,2.5
8,0.027623,0.029763,2.332711,9.7
9,0.050496,0.071250,4.889417,12.5


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   key                200000 non-null  int64  
 1   date               200000 non-null  object 
 2   fare_amount        200000 non-null  float64
 3   pickup_datetime    200000 non-null  object 
 4   pickup_longitude   200000 non-null  float64
 5   pickup_latitude    200000 non-null  float64
 6   dropoff_longitude  199999 non-null  float64
 7   dropoff_latitude   199999 non-null  float64
 8   passenger_count    200000 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 13.7+ MB


In [ ]:
# 4. Análisis y tratamiento de datos faltantes
# Comentario: Se identifican valores faltantes y se decide cómo tratarlos (eliminación, imputación, etc.).
# - df.isnull().sum()
# - Justificación de la decisión tomada

In [ ]:
# 5. Análisis y tratamiento de datos atípicos (outliers)
# Comentario: Se detectan y tratan valores atípicos en las variables numéricas.
# - Visualización con boxplots
# - Decisión sobre el tratamiento (eliminación, ajuste, etc.)

In [ ]:
# 6. Visualización de datos
# Comentario: Se grafican histogramas y scatterplots para analizar la distribución y relaciones entre variables.
# - Histograma de cada variable
# - Scatterplots entre variables relevantes
# - Diagramas de caja

In [ ]:
# 7. Codificación de variables categóricas (si corresponde)
# Comentario: Se codifican variables categóricas para su uso en modelos (OneHotEncoder, LabelEncoder, etc.).
# - Ejemplo: df['var'] = ...

In [ ]:
# 8. Matriz de correlación
# Comentario: Se calcula la matriz de correlación para analizar dependencias entre variables.
# - sns.heatmap(df.corr(), annot=True)

In [ ]:
# 9. Estandarización o escalado de datos
# Comentario: Se aplican técnicas de escalado para mejorar el desempeño de los modelos.
# - StandardScaler, MinMaxScaler, etc.

In [ ]:
# 10. División de datos: Train-Test (y opcionalmente Validación)
# Comentario: Se divide el conjunto de datos en entrenamiento y prueba.
# - from sklearn.model_selection import train_test_split
# - X_train, X_test, y_train, y_test = train_test_split(...)

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='fare_amount'), df['fare_amount'], test_size=0.2, random_state=42)

In [ ]:
X_train.describe()

In [ ]:
# 11. Implementación de modelos de regresión
# 11.a. Regresión lineal múltiple (LinearRegression)
# Comentario: Se ajusta el modelo base de regresión lineal.

In [ ]:
# 11.b. Métodos de gradiente descendente (SGDRegressor, otros)
# Comentario: Se prueban diferentes variantes de gradiente descendente.
# - Se grafican errores vs iteraciones (loss vs epochs)

In [ ]:
# 11.c. Métodos de regularización (Lasso, Ridge, Elastic Net)
# Comentario: Se aplican modelos con regularización y se varía el coeficiente.
# - Se comparan métricas para distintos valores de alpha.

In [ ]:
# 12. Evaluación de modelos: métricas de regresión
# Comentario: Se calculan las métricas elegidas para train y test (R2, MSE, RMSE, MAE, MAPE).
# - Justificación de las métricas utilizadas

In [ ]:
# 13. Gráficos de residuos
# Comentario: Se grafican los residuos para analizar el ajuste de los modelos.


In [ ]:
# 14. Análisis de fitting y conclusiones intermedias
# Comentario: Se analiza el fitting del modelo y se discuten los efectos de la regularización y gradiente descendente.


In [ ]:
# 15. Optimización y comparación de hiperparámetros
# Comentario: Se varían los hiperparámetros y se observa el efecto en el desempeño del modelo.
# - GridSearchCV, RandomizedSearchCV, o análisis manual


In [ ]:
# 16. Comparación de modelos
# Comentario: Se compara el desempeño de los distintos modelos y se selecciona el mejor.
# - Justificación de la métrica de comparación

In [ ]:
# 17. Conclusiones finales
# Comentario: Se redacta una conclusión sobre el trabajo realizado, principales hallazgos y aprendizajes.
